<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<CENTER>
<H1 style="color:red">
Accelerating (Numba) and Scaling (Dask) Python Codes on CPUs
</H1>
</CENTER>

# <font color='blue'>Reference Documents</font>
- <a href="http://numba.pydata.org/">Numba: A High Performance Python Compiler</a>
- <a href="https://examples.dask.org/applications/stencils-with-numba.html">Stencil Computations with Numba</a>
- <a href="http://deepdata.com.pl/numba.html">Python on steroids - speeding up calculations with numba</a>
- <a href="https://thedatafrog.com/en/articles/make-python-fast-numba/">Make python fast with numba</a>
- <a href="https://www.deeplearningwizard.com/deep_learning/production_pytorch/speed_optimization_basics_numba/">Speed Optimization Basics: Numba</a>
- <a href="https://flothesof.github.io/optimizing-python-code-numpy-cython-pythran-numba.html">Optimizing your code with NumPy, Cython, pythran and numba </a>

- <a href="https://docs.dask.org/en/latest/why.html">Why Dask?</a>
- <a href="https://github.com/dask/dask-tutorial">dask-tutorial</a>
- <a href="https://www.manning.com/books/data-science-with-python-and-dask">Data Science with Python and Dask</a>
- <a href="https://carpentries-incubator.github.io/lesson-parallel-python/aio/index.html">Parallel Programming in Python</a>
- <a href="https://www.youtube.com/watch?v=uGy5gT2vLdI&feature=youtu.be"> Working with the Python DASK library (video)</a>
- <a href="https://www.youtube.com/watch?v=t_GRK4L-bnw&feature=youtu.be">Who uses Dask (video)</a>

# <font color='blue'>What will be Covered?</font>

* Numba
   * What is Numba?
   * How Does Numba Work?
   * How to Use Numba?
   * Parallelization with Numba
* Dask
   * What is Dask?
   * Parallelize Code with `delayed`
   * Dask Array
   * Dask DataFrame
   * Dask Schedulers

#### Checking how many cores are available:

In [ ]:
import multiprocessing
print("Number of available cores: ", multiprocessing.cpu_count())

# <font color='blue'>Numba</font>

![fig_numba](https://thedatafrog.com/static/blog/images/2019/07/python_fast.0d88afcb4f8a.png)
Image Source: Lison Bernet 2019

## <font color='red'>What is Numba?</font>

> Numba is an open-source JIT compiler that translates a subset of Python and NumPy into fast machine code using `LLVM` (low-level virtual machine), via the llvmlite Python package. It offers a range of options for parallelising Python code for CPUs and GPUs, often with only minor code changes. 
>
>Wikipedia

- Numba is a Python open source package that was originally developed by Continuum Analytics.
- The core application area are math-heavy and array-oriented functions, which are in native Python pretty slow.
- From a function, Numba can generate native code for that function as well as the wrapper code needed to call it directly from Python. This compilation is done on-the-fly and in-memory.
- It accelerates Python code (numerical functions) for both CPU and GPU:
   - **Function Compiler**: Numba compiles Python functions, not whole applications or parts of it. It is a Python module meant to improve the performance of functions with the goal of achieving a speed comparable to `C`.
   - **Just-in-time**: (Dynamic translation) Numba translates the bytecode (intermediate code more abstract than the machine code) to machine code immediately before its execution to improve the execution speed.
   - **Numerically-focused**: Numba is focused on numerical data, such as int, float, complex. 
   
## <font color='red'>How Does Numba Work?</font>

- Assume that you have a function `do_math` that is decorated with the Numba `@jit` decorator. 
- Compilation will be deferred until the first function execution. 
- Numba will infer the argument types at call time, and generate optimized code based on this information. 
- Numba will also be able to compile separate specializations depending on the input types. 
- The diagram below, shows all the steps carried out by Numba to execute `do_math`. 

![fig_numba](https://miro.medium.com/max/1400/1*S0S4QUjR-BsdTICtT9797Q.png)
Image Source: Continuum Analytics

- **IR**: Intermediate Representations
- **Bytecode Analysis**: Intermediate code more abstract than machine code
- **LLVM**: Low Level Virtual Machine, infrastructure to develop compilers
- **NVVM**: It is an IR compiler based on LLVM, it is designed to represent GPU kernels

## <font color='red'>Numpy and Numba</font>
- One objective of Numba is having a seamless integration with NumPy. 
- Numba excels at generating code that executes on top of NumPy arrays.
- NumPy support in Numba comes in many forms:
    1. Numba understands calls to NumPy ufuncs (universal functions: there are over 60 of them) and is able to generate equivalent native code for many of them.
    2. NumPy arrays are directly supported in Numba.
    3. Numba is able to generate ufuncs and gufuncs (generalized universal functions). This means that it is possible to implement ufuncs and gufuncs within Python, getting speeds comparable to that of ufuncs/gufuncs implemented in C extension modules using the NumPy C API.
    
## <font color='red'>Usage</font>
- Numba provides several utilities for code generation.
- Its central feature is the `numba.jit()` decorator. 
- Using this decorator, you can mark a function for optimization by Numba’s JIT compiler. - - - Various invocation modes trigger differing compilation options and behaviours.


Consider using Numba if:

- Is numerically orientated.
- Uses Numpy
- Relies on loops

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import time
import numpy as np
import numba as nb
from numba import jit
from numba import njit
from numba import prange

**Checking your System**

The `numba -s` or `numba --sysinfo` command prints a lot of information about your system and your Numba installation and relevant dependencies.

In [ ]:
!numba -s

**Example**

Consider the function that multiplies two `nxn` matrices.

In [ ]:
def matrix_multiplication(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in range(n):
        for j in range(n):
            tmp = 0.
            for k in range(n):
                tmp  += A[i, k]*B[k, j]
            C[i, j] = tmp

In [ ]:
N = 200
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.zeros_like(A)
D = np.random.rand(N)

In [ ]:
tRegMat = %timeit -o matrix_multiplication(A, B, C)

There are two ways to use `Numba`:

- Method 1: As a function calling the function we want to speed
- Method 2: As a decorator of the function we want to speed

Method 1: Function

In [ ]:
numba_matrix_multiplication = jit(matrix_multiplication)

In [ ]:
tNumMat0 = %timeit -o numba_matrix_multiplication(A, B, C)

In [ ]:
print("Speedup Numba 0: {}".format(tRegMat.best/tNumMat0.best))

Method 2: Decorator

In [ ]:
@jit
def matrix_multiplication_numba(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in range(n):
        for j in range(n):
            tmp = 0.
            for k in range(n):
                tmp  += A[i, k]*B[k, j]
            C[i, j] = tmp

In [ ]:
%timeit matrix_multiplication_numba(A, B, C)

**Measuring the Performance of Numba**

- Once the compilation has taken place, Numba runs the machine code version of your function. 
- If it is called again with same argument types, it can reuse the cached version instead of having to compile again.
- A common mistake when measuring performance is not accounting for the above behaviour and to time code once with a simple timer that includes the time taken to compile your function in the execution time.

DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!

In [ ]:
@jit
def matrix_multiplication_numba(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in range(n):
        for j in range(n):
            tmp = 0.
            for k in range(n):
                tmp  += A[i, k]*B[k, j]
            C[i, j] = tmp

In [ ]:
start_1 = time.time()
matrix_multiplication_numba(A, B, C)
end_1 = time.time()
print("Elapsed (with compilation) = {}".format(end_1 - start_1))

NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE

In [ ]:
start_2 = time.time()
matrix_multiplication_numba(A, B, C)
end_2 = time.time()
print("Elapsed (after compilation) = {}".format(end_2 - start_2))

In [ ]:
print("Gain of time = {}".format((end_1 - start_1)/(end_2 - start_2)))

#### Function Signature

- You can specify the signature of the Numba function by describing the types of the arguments and the return type of the function. 
- This can produce slightly faster code as the compiler does not need to infer the types. 
- The drawback is that the function can no longer accept other types.

In [ ]:
def average_numbers(x, y):
    return (x + y)/2.0

In [ ]:
numba_average_numbers = jit(nb.float64(nb.int32, nb.int32))(average_numbers)

In [ ]:
@jit(nb.float64(nb.int32, nb.int32))
def average_numbers_numba(x, y):
    return (x + y)/2.0

- `nb.float64(nb.int32, nb.int32)` is the function’s signature specifying a function that takes two 32-bit integer arguments and returns a double precision float.
- You can also use the abbreviated notation: `nb.f8(nb.i4, nb.i4)`
- If you only pass `(nb.i4, nb.i4)` instead of `nb.f8(nb.i4, nb.i4)`, Numba will try to infer the type of the return value.
- Array signatures are specified by subscripting a base type according to the number of dimensions. 
     - A 1-dimension single-precision array would be written `nb.f4[:]`.
     - A 3-dimension array of the same underlying type would be `nb.f4[:,:,:]`.

In [ ]:
numba_matrix_multiplication = jit((nb.f8[:,:], nb.f8[:,:], nb.f8[:,:]))(matrix_multiplication)

In [ ]:
tNumMat1 = %timeit -o numba_matrix_multiplication(A, B, C)

In [ ]:
print("Speedup Numba 1: {}".format(tRegMat.best/tNumMat1.best))

**Another Example: Finding the Closet Two Points**

- Find the two closest points in an array of points in 2D. 
- Returns the two points, and the distance between them.
- If we have $N$ points, we would have to test NxN pairs of points. 
- This algorithm has a complexity of order $N \times N$, denoted $O(N \times N)$.

In [ ]:
import math
def python_closest(points):
    min_distance2 = 999999.
    mdp1, mdp2 = None, None
    for i in range(len(points)):
        for j in range(i+1, len(points)):
            distance2 = (points[i][0]-points[j][0])**2 + \
                        (points[i][1]-points[j][1])**2
            if distance2 < min_distance2:
                min_distance2 = distance2
                mdp1, mdp2 = points[i], points[j]
    return mdp1, mdp2, math.sqrt(min_distance2)

In [ ]:
points = np.random.uniform((-1,-1), (1,1), (8100,2))

In [ ]:
tcloset = %timeit -o python_closest(points)

We can now use Numba to speedup the calculations. We can explicitly pass the types of the arguments to have a better performance.

In [ ]:
import math

@jit('Tuple((float64[:], float64[:], float64))(float64[:,:])', nopython=True)
def numba_closest(points):
    min_distance2 = 999999.
    mdp1, mdp2 = None, None
    for i in prange(len(points)):
        for j in prange(i+1, len(points)):
            distance2 = (points[i][0]-points[j][0])**2 + \
                        (points[i][1]-points[j][1])**2
            if distance2 < min_distance2:
                min_distance2 = distance2
                mdp1, mdp2 = points[i], points[j]
    return mdp1, mdp2, math.sqrt(min_distance2)

In [ ]:
tcloset_numba = %timeit -o numba_closest(points)

In [ ]:
print("Speedup Closest Points: {}".format(tcloset.best/tcloset_numba.best))

### Compilation Options
A number of keyword-only arguments can be passed to the `@jit` decorator:
1. `nopython`: Numba has two compilation modes:
     - **nopython mode** (`nopython=True`): Compile the decorated function so that it will run entirely without the involvement of the Python interpreter. This mode produces the highest performance code, but requires that the native types of all values in the function can be inferred. Note that <font color="red">**`@njit`**</font> is an alias for <font color="red">**`@jit(nopython=True)`**</font>.
     - **object mode**: In this mode Numba will identify loops that it can compile and compile those into functions that run in machine code, and it will run the rest of the code in the interpreter. For best performance avoid using this mode!
     - By default Numba will automatically use **object mode** if **nopython mode** cannot be used for some reason. 
     - When you are in **nopython mode**, types that cannot be inferred by the compiler will generate an error.
2. `nogil`: 
     - Whenever Numba optimizes Python code to native code that only works on native types and variables (rather than Python objects), it is not necessary anymore to hold Python’s global interpreter lock (GIL). 
     - Numba will release the GIL when entering such a compiled function if you passed `nogil=True`.
     - When using `nogil=True`, you need to be wary of the usual pitfalls of multi-threaded programming (consistency, synchronization, race conditions, etc.).
3. `cache`:
     - To avoid compilation times each time you invoke a Python program, you can instruct Numba to write the result of function compilation into a file-based cache. 
     - This is done by passing `cache=True`.
4. `parallel`: 
     - Enables automatic parallelization (and related optimizations) for operations in the function known to have parallel semantics.
     - This feature is enabled by passing `parallel=True` and must be used in conjunction with `nopython=True`.

### Exercise

Use Numba (and Dask) to speed up the code below (calculations of `pi`):

In [ ]:
%%time

import random

def approximate_pi(num_samples):
    num_points_circ = 0

    for i in range(num_samples):
        # Select an arbitrary point in [-1,1]x[-1,1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point is inside the circle
        if x**2 + y**2 < 1.0:
            num_points_circ += 1

    return 4 * num_points_circ / num_samples

def mean(*args):
    return sum(args) / len(args)

num_samples = 10**6
num_experiments = 10

pi_approx = mean(*[approximate_pi(num_samples) for i in range(num_experiments)])

print("Approximation of Pi: {}".format(pi_approx))

<details><summary><b>Click here to access the solution</b></summary>
<p>


```python
%%time

import random

@nb.jit(nopython=True, nogil=True)
def nb_approximate_pi(num_samples):
    num_points_circ = 0

    for i in range(num_samples):
        # Select an arbitrary point in [-1,1]x[-1,1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point is inside the circle
        if x**2 + y**2 < 1.0:
            num_points_circ += 1

    return 4 * num_points_circ / num_samples

def mean(*args):
    return sum(args) / len(args)

num_samples = 10**6
num_experiments = 10

pi_approx = mean(*[np_approximate_pi(num_samples) for i in range(num_experiments)])

print("Approximation of Pi: {}".format(pi_approx))
```

</p>
</details>

### Fastmath
- In certain classes of applications strict IEEE 754 compliance is less important. 
- It is possible to relax some numerical rigour with view of gaining additional performance. 
- The way to achieve this behaviour in Numba is through the use of the `fastmath` keyword argument.

In [ ]:
@njit(fastmath=False)
def do_sum(A):
    acc = 0.
    # without fastmath, this loop must accumulate in strict order
    for x in A:
        acc += np.sqrt(x)
    return acc

@njit(fastmath=True)
def do_sum_fast(A):
    acc = 0.
    # with fastmath, the reduction can be vectorized as floating point
    # reassociation is permitted.
    for x in A:
        acc += np.sqrt(x)
    return acc

In [ ]:
time_do_sum = %timeit -o acc1 = do_sum(D)

In [ ]:
time_do_sum_fast = %timeit  -o acc2 = do_sum_fast(D)
print(time_do_sum.best / time_do_sum_fast.best)

## <font color="red"> Parallelization </font>

- The setting `parallel=True` in `jit()` enables a Numba transformation pass that attempts to automatically parallelize and perform other optimizations on (part of) a function.
- A user program may contain operations (for instance adding a scalar value to an array) that are known to have parallel semantics.
- Each operation could be parallelized individually but that might light to poor performance due to poor cache behavior.
- Numba uses instead auto-parallelization where it identifies all operations with parallel semantics and fuses adjacent ones together, to form one or more kernels that are automatically run in parallel.
- The process is fully automated without modifications to the user program.

### Explicit Parallel Loops

- Numba parallel execution also has support for explicit parallel loop declaration similar to that in OpenMP. 
- To indicate that a loop should be executed in parallel the `numba.prange` function should be used.
- This function behaves like Python `range` and if `parallel=True` is not set it acts simply as an alias of `range`. 
- Loops induced with `prange` can be used for embarrassingly parallel computation and also reductions.

**Example**

In [ ]:
@njit(parallel=True)
def matrix_multiplication_numba2(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in prange(n):
        for j in prange(n):
            tmp = 0.
            for k in prange(n):
                tmp += A[i, k]*B[k, j]
            C[i,j] = tmp

In [ ]:
tNumMat2 = %timeit -o matrix_multiplication_numba2(A, B, C)

In [ ]:
print("Speedup Numba 2: {}".format(tRegMat.best/tNumMat2.best))

**Another Example**

In [ ]:
def evaluate_functions(n):
    """
        Evaluate the trigononmetric functions for n values evenly
        spaced over the interval [-1500.00, 1500.00]
    """
    vector1 = np.linspace(-1500.00, 1500.0, n)
    iterations = 10000
    for i in range(iterations):
        vector2 = np.sin(vector1)
        vector1 = np.arcsin(vector2)
        vector2 = np.cos(vector1)
        vector1 = np.arccos(vector2)
        vector2 = np.tan(vector1)
        vector1 = np.arctan(vector2)

In [ ]:
@njit(parallel=True)
def evaluate_functions_numba(n):
    """
        Evaluate the trigononmetric functions for n values evenly
        spaced over the interval [-1500.00, 1500.00]
    """
    vector1 = np.linspace(-1500.00, 1500.0, n)
    iterations = 10000
    for i in prange(iterations):
        vector2 = np.sin(vector1)
        vector1 = np.arcsin(vector2)
        vector2 = np.cos(vector1)
        vector1 = np.arccos(vector2)
        vector2 = np.tan(vector1)
        vector1 = np.arctan(vector2)

In [ ]:
tRegFun = %timeit -o evaluate_functions(100)

In [ ]:
tNumFun = %timeit -o evaluate_functions_numba(100)

In [ ]:
print("Speedup: {}".format(tRegFun.best/tNumFun.best))

### Diagnostics
- We can produce diagnostic information about the transforms undertaken in automatically parallelizing the decorated code. 
- This information can be accessed in two ways:
     1. Setting the environment variable: `NUMBA_PARALLEL_DIAGNOSTICS`
     2. Calling the function `parallel_diagnostics()`
- The level of verbosity in the diagnostic information is controlled by an integer argument of value between 1 and 4 inclusive, 1 being the least verbose and 4 the most.

For additional information, consult the webpage: <a href="http://numba.pydata.org/numba-doc/latest/user/parallel.html"> http://numba.pydata.org/numba-doc/latest/user/parallel.html</a>.

In [ ]:
evaluate_functions_numba.parallel_diagnostics(level=4)

## <font color="red"> Calling Other Functions</font>

- Numba functions can call other Numba functions. 
- Both functions must have the `@jit` decorator, otherwise the code will be much slower.

```python
@jit
def square(x):
    return x ** 2

@jit
def hypot(x, y):
    return math.sqrt(square(x) + square(y))
```

## <font color="red">Numba and Pandas</font>

- Pandas is built on top of Numpy.
- Pandas offers flexibility in manipulating data but not necessary speed.
- This flexibility allows the creation of built-in function.
- Crude looping (over DataFrame rows for instance) in Pandas does not take advantage of any built-in optimizations, making it extremely inefficient.
- Using vectorized Pandas built-in functions (acting on Pandas Series) is almost always preferable to accomplishing similar ends with custom-written looping.

### Example

- We have a Pandas DataFrame and we want to add a new column by multiplying an exiting column by a constant.
- We use three methods methods for the multiplication operations: `apply` method, Pandas and vectorization with Numba.

In [ ]:
import pandas as pd

def multiply(x):
    return x * 5
    
@nb.vectorize
def multiply_numba(x):
    return x * 5

Create a table of 100,000 rows and 4 columns filled with random numbers from 0 to 100:

In [ ]:
df = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)),columns=['a', 'b', 'c', 'd'])
df

In [ ]:
time_apply = %timeit -o df['new_col1'] = df['a'].apply(multiply)

In [ ]:
time_pandas = %timeit -o df['new_col2'] = df['a'] * 5

In [ ]:
time_numba1 = %timeit -o df['new_col3'] = multiply_numba(df['a'].to_numpy())

In [ ]:
print("Multiply Apply:  {}".format(time_apply.best/time_apply.best))
print("Multiply Pandas: {}".format(time_apply.best/time_pandas.best))
print("Multiply Numba:  {}".format(time_apply.best/time_numba1.best))

#### Example

- Square the values of each row and take their mean to create a new column

In [ ]:
def square_mean(row):
    row = np.power(row, 2)
    return np.mean(row)

@njit
def square_mean_numba(arr):
    res = np.empty(arr.shape[0])
    arr = np.power(arr, 2)
    for i in prange(arr.shape[0]):
        res[i] = np.mean(arr[i])
    return res

In [ ]:
nrows_list = [10, 100, 1000, 10000, 100000]

In [ ]:
pandas_times = list()
for nrows in nrows_list:
    df = pd.DataFrame(np.random.randint(0,100,size=(nrows, 2)),columns=['a', 'b'])
    tp = %timeit -o df['new_col'] = df.apply(square_mean, axis=1)
    pandas_times.append(tp.best)

In [ ]:
numba_times = list()
for nrows in nrows_list:
    df = pd.DataFrame(np.random.randint(0,100,size=(nrows, 2)),columns=['a', 'b'])
    tn = %timeit -o df['new_col'] = square_mean_numba(df.to_numpy())
    numba_times.append(tn.best)

In [ ]:
print(pandas_times)
print(numba_times)

In [ ]:
import matplotlib.pyplot as plt

x = np.arange(len(nrows_list))

fig, axes = plt.subplots(nrows=1, ncols=1)
width = 0.25
axes.bar(x, pandas_times, width, label='Pandas apply')
axes.bar(x + width, numba_times, width, label='Numba function')
axes.set_xticks(x)
axes.set_xticklabels(nrows_list)
axes.legend(prop={'size': 10})
axes.set_yscale('log')
axes.set_xlabel('Number of rows')
axes.set_ylabel("Time (s)")

**Could we claim that Numpy/Numba is faster than Pandas?**

- Not necessarily!
- Over time, Pandas relies more on  Cython operations.
- In Pandas 1.0 (and newer versions) Pandas’ `apply()` method (applies a function along a specific axis of a DataFrame) can make use of Numba (if installed) instead of cython and be faster. 

## <font color="red">Things to Consider when Using Numba</font>

- Numba allows its behaviour to be changed through the use of <a href="http://numba.pydata.org/numba-doc/latest/reference/envvars.html">environment variables</a>. Unless otherwise mentioned, those variables have integer values and default to zero.
- Not all the <a href="http://numba.pydata.org/numba-doc/latest/reference/pysupported.html">Python features</a> are supported by Numba.
- While Python has arbitrary-sized integers, integers in Numba-compiled functions get a fixed size through type inference (usually, the size of a machine integer). This means that arithmetic operations can wrapround or produce undefined results or overflow.
- When Numba compiles machine code for functions, it treats global variables as constants to ensure type stability.
- Numba may or may not copy global variables referenced inside a compiled function. Small global arrays are copied for potential compiler optimization with immutability assumption. However, large global arrays are not copied to conserve memory. The definition of “small” and “large” may change.
- Numba does not work with recusive function.
- For some operations, Numba may use a different algorithm than Python or Numpy. The results may not be bit-by-bit compatible. The difference should generally be small and within reasonable expectations. However, small accumulated differences might produce large differences at the end, especially if a divergent function is involved.


## <font color="red">[Steps for Getting the Best out of Numba](https://techdecoded.intel.io/resources/parallelism-in-python-using-numba/#gs.m9ly7s)</font>

Achieving parallelism in Python with Numba is about knowing a few fundamentals and modifying your workflow to take these methods into account while you’re actively coding in Python. Here are the steps in the process:

1. **Ensure the abstraction of your core kernels is appropriate**. Numba requires the optimization target to be in a function. Unnecessarily complex code can cause the Numba compilation to fall back to object code.
2. **Look for places in your code where you see processing data in some form of a loop with a known data type**. Examples would be a for-loop iterating over a list of integers, or an arithmetic computation that processes an array in pure Python.
3. **If you’re using NumPy and SciPy, look at computations that can be stacked in a single statement and that are not BLAS or LAPACK functions**. These are prime candidates for using the ufunc optimization capabilities of Numba.
4. **Experiment with Numba’s compilation options**.
5. **Determine the intended datatype signature of the function and core code**. If it’s known (such as int8 or int32), then inform Numba about which input datatype parameters it should expect.

# <font color="blue">Dask</font>

![fig_dask](https://miro.medium.com/max/1000/1*D6mSsdWECFLn6wJne4VTjg.png)


## <font color="red"> What is Dask?</font>

- A flexible library for parallel computing in Python that makes it easy to build intuitive workflows for ingesting and analyzing large, distributed datasets. 
- A native parallel analytics tool designed to integrate seamlessly with Numpy, Pandas, and Scikit-Learn. 
- An out-of-core (data is read into memory from disk on an as-needed basis) parallelization library that seamlessly integrates with existing NumPy and Pandas data structures to address the following:
     * **The available dataset does not fit in memory of a single machine.**
     * **The data processing task is time consuming and needs to be sped up.**
- Orchestrates parallel threads or processes for us and help speed up processing times.

Dask consists of several different components and APIs, which can be categorized into three layers: the scheduler, low-level APIs, and high-level APIs.

- Dask provides a few high-level constructs called Dask Bags, Dask DataFrames, and Dask Arrays. They provide an easy-to-use interface to parallelize many of the typical data transformations in ML workflows. 
- Dask allows the creation of highly customized job execution graphs by using their extensive Python API (e.g., `dask.delayed`) and integration with existing data structures.


![fig_layers](http://bicortex.com/bicortex/wp-content/post_content//2019/06/Dask_APIs_Architecture.png)
Image Source: bicortex.com


The diagram below describes the steps Dask takes to manipulate data.

- The operation is broken down into a sequence of operations on smaller partitions of our data (without having to read the whole dataset into memory).
- Dask reads each partition as it is needed and computes the intermediate results. 
- The intermediate results are aggregated into the final result.
- Dask handles all of that sequencing internally for us. 
- On a single machine, Dask can use threads or processors to parallelize these operations. 

![fig_proc](https://www.manifold.ai/hs-fs/hubfs/Blog%20Post%20Illos/ML%20pipelines%20-%20dask%20single%20machine.jpeg?width=600&name=ML%20pipelines%20-%20dask%20single%20machine.jpeg)
Image Source: www.manifold.ai


**Advantages of Using Dask**

- Fully implemented in Python and natively scales NumPy, Pandas, and scikit-learn.
- Can be used effectively to work with both medium datasets on a single machine and large datasets on a cluster.
- Can be used as a general framework for parallelizing most Python objects.
- Has a very low configuration and maintenance overhead.



>Dask provides high-level Array, Bag, and DataFrame collections that mimic NumPy, lists, and Pandas but can operate in parallel on datasets that don’t fit into main memory. Dask’s high-level collections are alternatives to NumPy and Pandas for large datasets.

In [ ]:
#!python -m pip install dask[dataframe] --upgrade

In [ ]:
#!pip install memory_profiler

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import dask
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 

In [ ]:
from memory_profiler import memory_usage
import memory_profiler
%load_ext memory_profiler

## <font color="red"> Parallelize Code with `dask.delayed`</font>

- A simple way to parallelize the code.
- Allows users to delay function calls into a task graph with dependencies.
- Systems like `dask.dataframe` are built with `dask.delayed`.

**Simple Example**

Consider the following functions:

In [ ]:
import time

def increment(x):
    time.sleep(1.0)
    return x + 1

def double(x):
    time.sleep(1.0)
    return 2 * x

def add(x, y):
    time.sleep(1.0)
    return x + y

In [ ]:
%%time

x = increment(1)
y = increment(2)
z = add(x, y)

- We use the `dask.delayed` decorator to parallelize the functions `increment` and `add`.
- By decorating the functions, we record what we want to compute as tasks into graphs that will be run later on parallel hardware.

In [ ]:
xd = dask.delayed(increment)(1)
yd = dask.delayed(increment)(2)
zd = dask.delayed(add)(xd, yd)
zd

- When we call the delayed version by passing the arguments, exactly as before, but the original function isn't actually called yet.
- A delayed object is made, which keeps track of the function to call and the arguments to pass to it.
- We use the `visualize` method (relies on the `graphviz` package) that provide a visual representation of the operations being performed.

In [ ]:
zd.visualize(rankdir='LR')

- Note that we have not physically calculated **total** yet.
- We need to apply the `compute` method to get the answer. 
- <font color="red">It is only here that the data are loaded into memory for calculations</font>.
- The calculations are done through using a local thread pool.

In [ ]:
%%time
dask.compute(zd)

**Using `delayed` in Loops**

Consider the sequential code with two for-loops:

In [ ]:
%%time

n = 7
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = increment(x)
    z = double(y)
    out.append(z)
    
total = 0
for z in out:
    total = add(total, z)

total

We can parallelize the above using the `delayed` decorator:

In [ ]:
%%time

n = 7
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = dask.delayed(increment)(x)
    z = dask.delayed(double)(y)
    out.append(z)
    
totald = 0
for z in out:
    totald = dask.delayed(add)(totald, z)

totald

We can also get the visual representation through a task graph.

In [ ]:
totald.visualize()

In [ ]:
%%time
dask.compute(totald)

## Exercise 

Use the `delayed` decorator to parallelize the code below:

In [ ]:
def is_odd(x):
    return x%2

In [ ]:
%%time

n = 10
data = [i+1 for i in range(n)]

results = list()

for x in data:
    if is_odd(x):
        y = double(x)
    else:
        y = increment(x)
    results.append(y)

total = sum(results)
print(total)

<details><summary><b>Click here to access the solution</b></summary>
<p>


```python
%%time

n = 10
data = [i+1 for i in range(n)]

results = list()

for x in data:
    if is_odd(x):
        y = dask.delayed(double)(x)
    else:
        y = dask.delayed(increment)(x)
    results.append(y)

total = sum(results)
print(total.compute())
```

</p>
</details>

**<font color="red">Important Lessons</font>**

- The `delayed` decorator adds overhead.
- It is good not to use it when a task requires a little amount of time.
- Call `delayed` on the function not the result.
- Break up computations into many pieces. You achieve parallelism by having many delayed calls, not by using only a single one: Dask will not look inside a function decorated with `delayed` and parallelize that code internally.

## <font color="red"> Dask Array</font>

- Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid. 
    - _Parallel_: Uses all of the cores on your computer
    - _Larger-than-memory_: Lets you work on datasets that are larger than your available memory by breaking up your array into many small pieces, operating on those pieces in an order that minimizes the memory footprint of your computation, and effectively streaming data from disk.
    - _Blocked Algorithms_: Perform large computations by performing many smaller computations
- They support a large subset of the Numpy API.

![fig_array](https://miro.medium.com/max/1388/1*JfQnXJ5_R104bPyE8_XhwQ.png)

**Create a Dask Array**

- Create a 20000x20000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). 
- There are 400 (20x20) numpy arrays of size 1000x1000.

In [ ]:
x = da.random.random((20000, 20000), chunks=(1000, 1000))
x

In [ ]:
print("Shape: ",  x.shape)
print("Size:  ",  x.size)
print("Chunks: ", x.chunks)

We can use Numpy syntax:

In [ ]:
y = x + x.T
y.shape

In [ ]:
mu = x.mean(axis=0)
mu

In [ ]:
z = y[::2, 5000:].mean(axis=1)
z

In [ ]:
z.visualize(rankdir="LR")

Use the **`compute()`** function if you want your result as a NumPy array.

In [ ]:
mu[0].compute()

In [ ]:
w = z.compute()
print(type(w), w.shape )

**Persit Data in Memory**

- If you have the available RAM for your dataset then you can persist data in memory.
- This allows future computations to be much faster.

In [ ]:
%time y.sum().compute()

In [ ]:
y = y.persist()

In [ ]:
%time y[0, 0].compute()

In [ ]:
%time y.sum().compute()

**Numpy against Dask**

In [ ]:
def f_numpy():
    x = np.random.normal(10, 0.1, size=(20000, 20000)) 
    y = x.mean(axis=0)[::100]

In [ ]:
%%memit
f_numpy()

In [ ]:
%%time
f_numpy()

In [ ]:
def f_dask():
    x = da.random.normal(10, 0.1, size=(20000, 20000), 
                         chunks=(1000, 1000))
    y = x.mean(axis=0)[::100].compute() 

In [ ]:
%%memit
f_dask()

In [ ]:
%%time
f_dask()

Reshapping the chunk size might provide a better performance:

In [ ]:
def f_dask2():
    x = da.random.normal(10, 0.1, size=(20000, 20000), 
                         chunks=(10000, 100))
    y = x.mean(axis=0)[::100].compute() 

In [ ]:
%%time
f_dask2()

**Dask finished faster, but used more total CPU time because Dask was able to transparently parallelize the computation because of the chunk size.**

**<font color="red">Things to Consider</font>**

- If your data fits in RAM and you are not performance bound, then using NumPy might be the right choice. Dask adds another layer of complexity which may get in the way.
- If you are just looking for speedups rather than scalability then you may want to consider using Numba for manipulating Numpy arrays.
- How to select the chunk size?
     - Too small: huge overheads.
     - Poorly aligned with data: inefficient reading.
     - Recommended to have a chuck size of at least 100 Mb.
     - Choose a chunk size that is large in order to reduce the number of chunks that Dask has to think about (which affects overhead) but also small enough so that many of them can fit in memory at once. Dask will often have as many chunks in memory as twice the number of active threads.
   

**Avoid Oversubscribing Threads**
     
- By default Dask will run as many concurrent tasks as you have logical cores. 
- It assumes that each task will consume about one core.
- Many array-computing libraries (used in Dask) are themselves multi-threaded, which can cause contention and low performance.
- For better performance, we need to explicitly specify the use of one thread:

```python
   export OMP_NUM_THREADS=1
   export MKL_NUM_THREADS=1
   export OPENBLAS_NUM_THREADS=1
```

## <font color="red"> Dask DataFrames</font>

- Pandas is great for tabular datasets that fit in memory. 
- Dask becomes useful when the dataset you want to analyze is larger than your machine's RAM. 
- Dask DataFrames:
     - Coordinate many Pandas DataFrames, partitioned along an index. 
     - Support a large subset of the Pandas API.
- One operation on a Dask DataFrame triggers many Pandas operations on the constituent pandas DataFrames in a way that is mindful of potential parallelism and memory constraints.
- Some of the operations that are really fast if you use Dask Dataframes:
     - Arithmetic operations (multiplying or adding to a Series)
     - Common aggregations (`mean`, `min`, `max`, `sum`, etc.)
     - Calling `apply`
     - Calling `value_counts()`, `drop_duplicates()` or `corr()`
     - Filtering with `loc`, `isin`, and row-wise selection

![fig_df](https://pythondata.com/wp-content/uploads/2016/11/Screen-Shot-2016-11-24-at-6.52.24-PM-168x300.png)

## <font color="red"> Schedulers</font>

- After Dask generates the task graphs, it needs to execute them on parallel hardware. 
- It is the role of a task scheduler. 
- There are different task schedulers. Each will consume a task graph and compute the same result, but with different performance characteristics.

![schedulers](https://docs.dask.org/en/latest/_images/dask-overview.svg)

Image Source: [https://docs.dask.org/en/latest/](https://docs.dask.org/en/latest/)

To execute the task graphs there are two types of schedulers:
* **Single machine**: Provides basic features on a local process or thread pool. It is simple and cheap to use, although it can only be used on a single machine and does not scale
* **Distributed**: Offers more features, but also requires a bit more effort to set up. It can run locally or distributed across a cluster.

### <font color="green"> Single Machine Scheduler</font>

Consider the following example:

In [ ]:
%%time

n = 7
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = dask.delayed(increment)(x)
    z = dask.delayed(double)(y)
    out.append(z)
    
totald = 0
for z in out:
    totald = dask.delayed(add)(totald, z)

totald

In [ ]:
import multiprocessing
print("Number of available processors: ", multiprocessing.cpu_count())

**Single thread**

- The single-threaded synchronous scheduler executes all computations in the local thread with no parallelism at all.
- It is useful for debugging.

In [ ]:
%time totald.compute(scheduler='synchronous')

**Local threads**

- Uses `multiprocessing.pool.ThreadPool`

To use all the available processors:

In [ ]:
%time totald.compute(scheduler='threads')

To use some of the processors:

In [ ]:
%time totald.compute(scheduler='threads', num_workers=2)

**Local processes**

- The multiprocessing scheduler executes computations with a local `multiprocessing.Pool`.
- Every task and all of its dependencies are shipped to a local process, executed, and then their result is shipped back to the main process. 
- Moving data to remote processes and back can introduce performance penalties, particularly when the data being transferred between processes is large. 
- The multiprocessing scheduler is an excellent choice when workflows are relatively linear, and so does not involve significant inter-task data transfer as well as when inputs and outputs are both small, like filenames and counts.

To use all the available processors:

In [ ]:
%time result = totald.compute(scheduler='processes')

To use some of the processors:

In [ ]:
%time result = totald.compute(scheduler='processes', num_workers=2)

### <font color="green">Distributed Scheduler</font>

- The Dask distributed scheduler can either be setup on a cluster or run locally on a personal machine. 
- It is a centrally managed, distributed, dynamic task scheduler. 
     - The central dask-scheduler process coordinates the actions of several dask-worker processes spread across multiple machines and the concurrent requests of several clients.
     - The scheduler is asynchronous and event-driven, simultaneously responding to requests for computation from multiple clients and tracking the progress of multiple workers.
     - The event-driven and asynchronous nature makes it flexible to concurrently handle a variety of workloads coming from multiple users at the same time while also handling a fluid worker population with failures and additions. 
     - Workers communicate amongst each other for bulk data transfer over TCP.
- To set up `dask.distributed`, we need to create client instance by calling `Client` class from `dask.distributed`. 
- It will internally create a dask scheduler and dask workers. 
- We will get the **link of the dashboard** where we can analyze tasks running in parallel. 
- We can pass a number of workers (using the `n_workers` argument) and threads to use per worker process (using the `threads_per_worker` argument).
- As soon as you create a client, Dask will automatically start using it.

In [ ]:
from dask.distributed import Client
client = Client()
#client = Client(n_workers=2, threads_per_worker=4)
client.cluster

If you aren’t in jupyterlab and using the `dask-labextension`, you can  click the `Dashboard` link to open up the diagnostics dashboard.

In [ ]:
import random

def random_slow_add(x, y):
    time.sleep(random.randrange(3,10))
    return x + y

In [ ]:
results = []

for x in data:
    y = dask.delayed(random_slow_add)(x, 1)
    results.append(y)
    
total = dask.delayed(sum)(results)

In [ ]:
%time result = total.compute()
result

Shut down the cluster:

In [ ]:
client.close()

**<font color="red">Things to Consider</font>**

- Each Dask task has overhead (about 1 ms). If you have a lot tasks this overhead can add up. It is a good idea to give each task more than a few seconds of work.
- To better understand how your program is performing, check the [Dask Performance Diagnostics](https://distributed.dask.org/en/latest/diagnosing-performance.html) documentation. You can also view the [video](https://docs.dask.org/en/stable/diagnostics-distributed.html) to find out how to group your work into fewer, more substantial tasks. This might mean that you call lazy operations at once instead of individually. This might also repartitioning your dataframe(s).
- A good rule of thumb for choosing number of threads per Dask worker is to choose the square root of the number of cores per node. 
     - In general more threads per worker are good for a program that spends most of its time in NumPy, SciPy, Numba, etc., and fewer threads per worker are better for simpler programs that spend most of their time in the Python interpreter.
- The Dask scheduler runs on a single thread, so assigning it its own node is a waste.
- There is no hard limit on Dask scaling. The task overhead though will eventually start to swamp your calculation depending on how long each task takes to compute. 

## <font color="red">Parallelize using Dask `Map_Partition`</font>

We construct a Dask DataFrame from pandas dataframe using `from_pandas` function and specify the number of partitions (`nparitions`) to break this dataframe into.

```python
   dd = ddf.from_pandas(df, npartitions=N)
```

`ddf` is the name you imported Dask Dataframes with, and `npartitions` is an argument telling the Dataframe how you want to partition it.

Each partition will run on a different thread, and communication between them will become too costly if there are too many.

Let us build a Pandas DataFrame with 100000 rows and two columns with values selected randomly between 1 and 1000.

In [ ]:
num_rows = 100000
df = pd.DataFrame({'X':np.random.randint(1000, size=num_rows),
                   'Y':np.random.randint(1000, size=num_rows)})
df

We break the DataFrame into 4 partitions (or the number of available cores):

In [ ]:
ddf = dd.from_pandas(df, npartitions=4)

We will apply `add_squares` method on each of these partitions:

In [ ]:
def add_squares(df):
    return df.X**2 + df.Y**2

In [ ]:
%%time

ddf['add_squares'] = ddf.map_partitions(add_squares, 
                               meta=(None, 'int64')).compute()

In [ ]:
ddf

# <font color="blue">Numba and Dask</font>

- Numba allows for run-time compilations of functions to optimize single-machine code.
    - If you intend to call a function multiple times, you can decrease your compute time significantly by compliling the function on the first call. 
    - Numba is useful for speeding up individual tasks.
- Dask is a parallel computing library for out-of-memory and distributed computations. 
    - At the heart of dask are a series of task schedulers — algorithms for determining when and how to run various user-defined computational “tasks”; consequently, dask can automatically identify which tasks can be run in parallel, or not run at all. 
    - Employing dask’s schedulers allows us to scale out to a network of many interrelated tasks and efficiently compute only those outputs we need, even on a single machine.
    


### <font color='green'>Example:</font> Manipulating Arrays

- We use Numpy or Dask arrays to find the appromation of Pi.
- We add Numba to find out if we can accelerate the calcultions.

![pi](https://miro.medium.com/max/2400/0*Hc8vWedSy1mTOuw6)
Image Source: [https://blog.esciencecenter.nl/parallel-programming-in-python-7fd62c90217d](https://blog.esciencecenter.nl/parallel-programming-in-python-7fd62c90217d)

Standard Python version:

In [ ]:
import random

def py_approximate_pi(num_samples):
    num_points_circ = 0

    for i in range(num_samples):
        # Select an arbitrary point in [-1,1]x[-1,1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point is inside the circle
        if x**2 + y**2 < 1.0:
            num_points_circ += 1

    return 4 * num_points_circ / num_samples

Numpy Version:

In [ ]:
def np_approximate_pi(num_samples):
    pts = np.random.uniform(-1, 1, (2, num_samples))
    
    # Count number of impacts inside the circle
    num_points_circ = np.count_nonzero((pts**2).sum(axis=0) < 1)

    return 4 * num_points_circ / num_samples

Numba version:

In [ ]:
@nb.jit(nopython=True, nogil=True)
def nb_approximate_pi(num_samples):
    num_points_circ = 0

    for i in range(num_samples):
        # Select an arbitrary point in [-1,1]x[-1,1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point is inside the circle
        if x**2 + y**2 < 1.0:
            num_points_circ += 1

    return 4 * num_points_circ / num_samples

Numba and Dask Version:

In [ ]:
@dask.delayed
@nb.jit(nopython=True, nogil=True)
def nbdk_approximate_pi(num_samples):
    num_points_circ = 0

    for i in range(num_samples):
        # Select an arbitrary point in [-1,1]x[-1,1]
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)

        # Check if the point is inside the circle
        if x**2 + y**2 < 1.0:
            num_points_circ += 1

    return 4 * num_points_circ / num_samples

We want to repeat the calculations several times and then take the verage value:

In [ ]:
def reg_mean(*args):
    return sum(args) / len(args)

In [ ]:
@dask.delayed
def dk_mean(*args):
    return sum(args) / len(args)

In [ ]:
number_samples = 10**6
number_experiments = 10

In [ ]:
time_py = %timeit -o reg_mean(*[py_approximate_pi(number_samples) for i in range(number_experiments)])

In [ ]:
time_np = %timeit -o reg_mean(*[np_approximate_pi(number_samples) for i in range(number_experiments)])

In [ ]:
time_nb = %timeit -o reg_mean(*[nb_approximate_pi(number_samples) for i in range(number_experiments)])

In [ ]:
time_nbdk = %timeit -o dk_mean(*[nbdk_approximate_pi(number_samples) for i in range(number_experiments)]).compute()

In [ ]:
print("Python:         {}".format(time_py.best/time_py.best))
print("Numpy:          {}".format(time_py.best/time_np.best))
print("Numba:          {}".format(time_py.best/time_nb.best))
print("Numba and Dask: {}".format(time_py.best/time_nbdk.best))

### <font color='green'>Example:</font> Manipulating DataFrames

We implment five tests to manipullate DataFrames in both Pandas and Dask:

1. Pandas and `apply` method
2. Pandas and Numba
3. Dask and `map_partitions` & `apply` methods
4. Dask, `map_partitions` & `apply` methods and Numba
5. Dask and `assign` method

In [ ]:
def check_dist(col1, col2, col3):
    """
       Check if the distance is less that a value.
    """
    dist = np.sqrt(col1**2 + col2**2)
    return dist < col3

@jit(nopython=True)
def check_dist_fast(col1, col2, col3):
    dist = np.sqrt(col1**2+col2**2)
    return dist < col3

Create a Pandas DataFrame:

In [ ]:
num_rows = 1000000
num_cols = 3
df = pd.DataFrame(np.random.random(size=(num_rows,num_cols))*100, 
                  columns=['col1','col2','col3'])

Create the corresponding Dask DataFrame:

In [ ]:
ddf = dd.from_pandas(df, npartitions=4)

#### Pandas and `apply` method

In [ ]:
t0 = time.time()
df['col4'] = df.apply(lambda x: check_dist(x.col1, x.col2, x.col3), 
                      axis=1)
time_py = time.time()-t0
print("Regular pandas took", time_py)
print('Number of True', df['col4'].sum())
df = df.drop('col4',axis=1)

#### Pandas and Numba

In [ ]:
t0 = time.time()
df['col4'] = check_dist_fast(df.col1.to_numpy(),
                             df.col2.to_numpy(),df.col3.to_numpy())
time_nbpd = time.time()-t0
print("Numba pandas took", time_nbpd)
print('Number of True', df['col4'].sum())
df = df.drop('col4',axis=1)

#### Dask and `map_partitions` & `apply` methods

In [ ]:
t0 = time.time()
ddf['col4'] = ddf.map_partitions(lambda d: d.apply(
    lambda x: check_dist(x.col1,x.col2,x.col3),axis=1)
                                ).compute()

time_dkpd = time.time()-t0
print("Dask pandas took", time_dkpd)
print('Number of True', ddf['col4'].sum().compute())
ddf = ddf.drop('col4',axis=1)

#### Dask, `map_partitions` & `apply` methods and Numba

In [ ]:
t0 = time.time()
ddf['col4'] = ddf.map_partitions(lambda d: d.apply(lambda x:
                    check_dist_fast(x.col1,x.col2,x.col3),axis=1)).compute()
time_dkmap = time.time()-t0
print("Numba Dask pandas took", time_dkmap)
print('Number of True', ddf['col4'].sum().compute())
ddf = ddf.drop('col4',axis=1)

#### Dask and  `assign` method

In [ ]:
t0 = time.time()
ddf = ddf.assign(col4=lambda x: check_dist(x.col1, x.col2, x.col3))
ddf.compute()
time_dkassign = time.time()-t0
print("Dask using Assign took", time_dkassign)
print('Number of True', ddf['col4'].sum().compute())
ddf = ddf.drop('col4',axis=1)

In [ ]:
print(f"Pandas/apply:               {time_py/time_py}")
print(f"Pandas + Numba:             {time_py/time_nbpd}")
print(f"Dask/map_partitions/apply:  {time_py/time_dkpd}")
print(f"Dask/map_partition + Numba: {time_py/time_dkmap}")
print(f"Dask/assign:                {time_py/time_dkassign}")